In [1]:
import requests
from io import BytesIO
import pandas as pd
import re
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats

In [2]:
plt.rc('font', family = 'Malgun Gothic')
plt.rc('axes', unicode_minus = False)
set_matplotlib_formats('retina')

# 다트 오픈 API 불러오기

In [3]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'}
cols = ['연결 포괄손익계산서','연결 재무상태표']
crtfc_key = '31718c7bf232574ee78e6f3f81c922043baad322'
corp_code ='005930'
company = '삼성전자'

## rcp_no 추출

In [4]:
url = 'https://opendart.fss.or.kr/api/list.xml?crtfc_key={}&corp_code={}&bgn_de=19990101&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003&page_count=100'.format(crtfc_key, corp_code)

In [5]:
res = requests.get(url, headers=headers)
page = res.content.decode('UTF-8')

In [6]:
period_list = re.findall(r'<report_nm>(.*?)</report_nm>', page)
rcp_no_list = re.findall(r'<rcept_no>(.*?)</rcept_no>', page)

In [7]:
dict(zip(period_list,rcp_no_list))

{'분기보고서 (2020.09)': '20201116001248',
 '반기보고서 (2020.06)': '20200814001766',
 '분기보고서 (2020.03)': '20200515001451',
 '사업보고서 (2019.12)': '20200330003851',
 '분기보고서 (2019.09)': '20191114001273',
 '반기보고서 (2019.06)': '20190814002218',
 '분기보고서 (2019.03)': '20190515001605',
 '사업보고서 (2018.12)': '20190401004781',
 '분기보고서 (2018.09)': '20181114001530',
 '반기보고서 (2018.06)': '20180814001113',
 '분기보고서 (2018.03)': '20180515001699',
 '사업보고서 (2017.12)': '20180402005019',
 '분기보고서 (2017.09)': '20171114001694',
 '반기보고서 (2017.06)': '20170814001832',
 '분기보고서 (2017.03)': '20170515003806',
 '사업보고서 (2016.12)': '20170331004518',
 '분기보고서 (2016.09)': '20161114001491',
 '반기보고서 (2016.06)': '20160816001779',
 '분기보고서 (2016.03)': '20160516003174',
 '사업보고서 (2015.12)': '20160330003536',
 '분기보고서 (2015.09)': '20151116000976',
 '반기보고서 (2015.06)': '20150817000859',
 '분기보고서 (2015.03)': '20150515001379',
 '사업보고서 (2014.12)': '20150331002915',
 '분기보고서 (2014.09)': '20141114000755',
 '반기보고서 (2014.06)': '20140814000743',
 '분기보고서 (201

## dcm_no 추출

In [8]:
dcm_no_list = []

for rcp_no in rcp_no_list:
    res = requests.get('http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp_no))
    page = res.content.decode('utf-8')
    dcm_no = re.findall(r"{}', '(.*?)',".format(rcp_no), page)[0]
    dcm_no_list.append(dcm_no)

## 데이터 다운로드

In [9]:
dcm_no_list

['7549051',
 '7446167',
 '7341446',
 '7202455',
 '6958001',
 '6846651',
 '6738798',
 '6616741',
 '6382016',
 '6282263',
 '6180129',
 '6060273',
 '5845320',
 '5752868',
 '5653406',
 '5540271',
 '5349987',
 '5261457',
 '5146351',
 '5026126',
 '4854164',
 '4770098',
 '4669876',
 '4556583',
 '4384460',
 '4299541',
 '4215971',
 '4114631',
 '3952512',
 '3875930',
 '3803552',
 '3714646',
 '3560249',
 '3493599',
 '3332300',
 '3422441',
 '3207134',
 '3151930',
 '3138683',
 '2965731',
 '3067153',
 '2831348',
 '2762561',
 '2591647',
 '2695124',
 '2455237',
 '2384079',
 '2191599',
 '2309913',
 '2055378',
 '2003746',
 '1864870',
 '1944550',
 '1751900',
 '1700372',
 '1574002',
 '1644600',
 '1471142',
 '1432656',
 '1429298',
 '1321907',
 '1383787',
 '1202395',
 '1156947',
 '1049370',
 '1104240',
 '947352',
 '908765',
 '809100',
 '863843',
 '703011',
 '660953',
 '570539',
 '618045',
 '582896',
 '469435',
 '431170',
 '353722',
 '396273',
 '263890',
 '237424',
 '178927',
 '206571',
 '113424',
 '89871',


In [20]:
def down_excel(period,rcp_no,dcm_no,company):
    url = 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={}&dcm_no={}&lang=ko'.format(rcp_no,dcm_no)
    resp = requests.get(url, headers = headers)
    table = BytesIO(resp.content)
    for sheet in cols:
        df = pd.read_excel(table, sheet_name = sheet, skiprows = 5)
        df.to_csv(str(period) + company + sheet + '.csv',encoding='utf-8-sig')

In [11]:
for period, rcp_no, dcm_no in zip(period_list, rcp_no_list, dcm_no_list):
    down_excel(period, rcp_no, dcm_no, company)

TypeError: expected str, bytes or os.PathLike object, not NoneType

# 불러온 데이터 분석

In [12]:
rcp_no_list = re.findall(r'<rcept_no>(.*?)</rcept_no>', page)

## 리스트 내 특정 문자를 갖는 문자열 추출

In [13]:
search = '포괄손익계산서'
data_list = os.listdir()

match_list = list()
for word in data_list:
    if search in word:
        match_list.append(word)
match_list

['[기재정정]반기보고서 (2011.06)삼성전자연결 포괄손익계산서.csv',
 '[첨부추가]사업보고서 (2011.12)삼성전자연결 포괄손익계산서.csv',
 '반기보고서 (2011.06)삼성전자연결 포괄손익계산서.csv',
 '반기보고서 (2012.06)삼성전자연결 포괄손익계산서.csv',
 '반기보고서 (2013.06)삼성전자연결 포괄손익계산서.csv',
 '반기보고서 (2014.06)삼성전자연결 포괄손익계산서.csv',
 '반기보고서 (2015.06)삼성전자연결 포괄손익계산서.csv',
 '반기보고서 (2016.06)삼성전자연결 포괄손익계산서.csv',
 '반기보고서 (2017.06)삼성전자연결 포괄손익계산서.csv',
 '반기보고서 (2018.06)삼성전자연결 포괄손익계산서.csv',
 '반기보고서 (2019.06)삼성전자연결 포괄손익계산서.csv',
 '반기보고서 (2020.06)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2011.09)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2012.03)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2012.09)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2013.03)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2013.09)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2014.03)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2014.09)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2015.03)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2015.09)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2016.03)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2016.09)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2017.03)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2017.09)삼성전자연결 포괄손익계산서.csv',
 '분기보고서 (2018.03)삼성전자연결 포괄손익계산서.csv',


## 전체 데이터, 포괄손익계산서에서  당기순이익(손실) 추출 

In [19]:
pl = {}

for filename in match_list:
    df = pd.read_csv('./{}'.format(filename), encoding='utf-8-sig')
    df_boolean = df == '당기순이익(손실)'
    x = df_boolean.sum(axis=1).values.argmax()
    y = df_boolean.sum(axis=0).values.argmax()
    num = df.iloc[x,y+1]
    pl[filename[7:14]] =  num

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 1: invalid start byte

In [ ]:
pl

In [ ]:
# 기간 정렬
sort = sorted(pl)[2:-2]

In [ ]:
# 키 값들 정렬
pl_new = {}

summation = 0
for i, period in zip(range(30),sort):
    remain = i%4
    if remain == 3:
        print(i, period, int(pl[period]) - summation)
        pl_new[period] = int(pl[period]) - summation
        summation = 0
    else:
        summation = summation + int(pl[period])
        print(i, period, pl[period])
        print(summation)
        pl_new[period] = int(pl[period])

## 시각화 예제 실습

In [ ]:
plt.plot([1, 2, 3, 4], [1,4,9,16])
plt.ylabel('some numbers')
plt.show()

In [ ]:
plt.plot([1, 2, 3, 4], [1, 4, 9, 16], 'ro') # r : 색 , o : marker
plt.axis([0, 6, 0, 20])
plt.show()

## 삼성전자 당기순이익 시각화

In [ ]:
pl_new.values()

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(pl_new.values())
plt.title('삼성전자', fontsize = 20)
plt.xlabel('기간',fontsize = 15)
plt.ylabel('당기순이익', fontsize = 15)
plt.xticks(np.arange(30), pl_new.keys(), rotation = 70)
plt.grid()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
# Fake dataset
height = [3, 12, 5, 18, 45]
bars = ('A', 'B', 'C', 'D', 'E')
y_pos = np.arange(len(bars))
 
# Create bars and choose color
plt.bar(y_pos, height, color = (0.5,0.1,0.5,0.6))
 
# Add title and axis names
plt.title('My title')
plt.xlabel('categories')
plt.ylabel('values')
 
# Limits for the Y axis
plt.ylim(0,60)
 
# Create names
plt.xticks(y_pos, bars)
 
# Show graphic
plt.show()
